## Analysis of blood glucose data to assess the reproducibility of the post-prandial blood glucose response (PPGR)
### Process the raw blood glucose data to generate 'glucose events' (initail nadir / peak / secondary nadir)

1. Preprocess the raw data
2. Timestamps matched for carbohydrate intake (i.e. all 35g CHO within 1 hour of eachother grouped together)
3. Use meal or insulin dosing to initiate a meal repsonse to food
4. Isolate the next 48 points (4-hour) data period to generate a PPGR
5. Split the PPGRs into breakfast, lunch or dinner data frames


### 'Cluster' or group the glucose events accoridng to clinically relevant parameters
1. 'Cluster' the glucose events based on the following set of rules
    * Meets correlation to all PPGRs in the cluster (>0.4 correlation score)
2. Al excluded PPGRs from the clusters to be clustered against themselves until all PPGRs are ultilized


### Import/load necessary libraries 

In [30]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import datetime
from datetime import timedelta
from statistics import mode
from scipy.signal import find_peaks

# Libraries for Correlations
from scipy.stats import pearsonr
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist

# Libraries for Prediction
#from statsmodels.tsa.arima_model import ARIMA
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LinearRegression
#from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
#from tensorflow.keras import layers
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler

import math
import pickle 
%matplotlib inline


### Load and preprocess necessary data sets

In [31]:
def read_data(filename):
    unfiltered = pd.read_csv(os.path.join("CSV Files", filename))
    unfiltered['glucose_level_ts'] = pd.to_datetime(unfiltered['glucose_level_ts'], dayfirst=True)
    unfiltered['meal_ts'] = pd.to_datetime(unfiltered['meal_ts'], dayfirst=True)
    unfiltered['bolus_ts'] = pd.to_datetime(unfiltered['bolus_ts'], dayfirst=True)
    return unfiltered


In [32]:
def slice_data(data, time_step, n_samples):
    timestamp = np.array(data['glucose_level_ts'], dtype=float)
    glucose = np.array(data['glucose_level'], dtype=float)
    n_samples_per_segment = int(time_step / timedelta(minutes=1)) * n_samples
    glucose_seg_ts = [timestamp[i:i + n_samples_per_segment] for i in range(0, len(data), n_samples_per_segment)]
    glucose_sliced = [glucose[i:i + n_samples_per_segment] for i in range(0, len(data), n_samples_per_segment)]
    return glucose_sliced, glucose_seg_ts


In [33]:
unfiltered = read_data('UoM2314.csv')


In [34]:
unfiltered.sort_values('glucose_level_ts', inplace=True)
unfiltered.reset_index(drop=True, inplace=True)


In [35]:
class EatingBolusEvent:
    def __init__(self,timestamp, type):
        self.eventType = type
        self.timestamp = timestamp


In [36]:
def interleave_arrays_increasing(meal, bolus):
    result = []
    i, j = 0, 0
    
    while i < len(meal) and j < len(bolus):
        if meal[i] < bolus[j]:
            result.append(("meal",meal[i]))
            i += 1
        else:
            result.append(("bolus",bolus[j]))
            j += 1
    return result 


In [37]:
def glucoseForMealsTs(glucose_ts_array, meal_ts):
    closest_values = []
    for element in meal_ts:
        # Find the index of the closest value in the comparison array
        closest_index = np.abs(glucose_ts_array - element).argmin()
        
        # Add the closest value to the closest_values array
        closest_values.append(glucose_ts_array[closest_index])

    return np.array(closest_values)
    

In [38]:
def glucoseForEventsTs(glucose_ts_array, events_ts):
    closest_values = []
    for _, event_ts in events_ts:
        # Find the index of the closest value in the comparison array
        closest_index = np.abs(glucose_ts_array - event_ts).argmin()
        
        # Get the closest value
        closest_value = glucose_ts_array[closest_index]

        # Add a tuple of (event_ts, closest_value) to the closest_values array
        closest_values.append((event_ts, closest_value))

    return closest_values
    

In [39]:
def bolusMealSeparation(meal_ts, bolus_start_ts, bolus_dose_filtered):
    viableBolusTimes = []
    
    for i in range(min(len(bolus_start_ts), len(bolus_dose_filtered))):
        isWithinRange = False
        
        for j in range(len(meal_ts)):
            if meal_ts[j] - pd.Timedelta(minutes=4) <= bolus_start_ts[i] <= meal_ts[j] + pd.Timedelta(hours=4):
                isWithinRange = True
                break
        
        if not isWithinRange:
            viableBolusTimes.append((bolus_start_ts[i], bolus_dose_filtered[i]))

    return viableBolusTimes


In [40]:
#def bolusMealSeparation(meal_ts,bolus_start_ts,bolus_dose_filtered):
#    viableBolusTimes = []
#    i = 0
#    j = 0
#    isWithinRange = False  
#    for i in range(len(bolus_start_ts)):
#        isWithinRange = False
#        for j in range(len(meal_ts)):
#            if meal_ts[j] - pd.Timedelta(minutes=4) <= bolus_start_ts[i] <= meal_ts[j] + pd.Timedelta(hours=4):
#                isWithinRange = True
#                break
#        if not isWithinRange:
#            viableBolusTimes.append((bolus_start_ts[i],bolus_dose_filtered[i]))
#
#    return viableBolusTimes


In [41]:
def groupBolus(bolus_array):
    time_ranges = [
    ("6am-10am", datetime.time(6, 0), datetime.time(10, 0)),
    ("10am-2pm", datetime.time(10, 0), datetime.time(14, 0)),
    ("2pm-6pm", datetime.time(14, 0), datetime.time(18, 0)),
    ("6pm-10pm", datetime.time(18, 0), datetime.time(22, 0)),
    ]

    # Create a DataFrame with date and value columns
    date_array = [t[0] for t in bolus_array]
    values_array = [t[1] for t in bolus_array]
    
    df = pd.DataFrame({'Date': date_array, 'Value': values_array})
    result = {}
    unique_dates = df['Date'].dt.date.unique()

    # Iterate through each date
    for date in unique_dates:
        date_mask = df['Date'].dt.date == date  # Create a mask for the current date
        
        # Iterate through each time range
        for label, start_time, end_time in time_ranges:
            # Create a mask for the current time range
            time_mask = (df['Date'].dt.time >= start_time) & (df['Date'].dt.time < end_time)
            
            # Combine the date and time masks
            mask = date_mask & time_mask
            
            # Find the maximum value for the current time range and date
            max_value = df.loc[mask, 'Value'].max()
            
            # Store the result in the dictionary
            result[f"{date} - {label}"] = max_value

    # Print the results
    for label, max_value in result.items():
        print(f"{label}: MaxValue = {max_value}")
    

In [42]:
def groupBolus2(bolus_array):
    time_ranges = [
        ("6am-10am", datetime.time(6, 0), datetime.time(10, 0)),
        ("10am-2pm", datetime.time(10, 0), datetime.time(14, 0)),
        ("2pm-6pm", datetime.time(14, 0), datetime.time(18, 0)),
        ("6pm-10pm", datetime.time(18, 0), datetime.time(22, 0)),
    ]

    result = []

    # Convert the bolus_array to a DataFrame
    df = pd.DataFrame(bolus_array, columns=['Timestamp', 'Value'])

    # Group the data by date
    grouped = df.groupby(df['Timestamp'].dt.date)

    for date, group_data in grouped:
        daily_result = {'Date': date, 'TimeRanges': []}

        for label, start_time, end_time in time_ranges:
            time_mask = (group_data['Timestamp'].dt.time >= start_time) & (group_data['Timestamp'].dt.time < end_time)

            max_value = group_data.loc[time_mask, 'Value'].max()
            max_timestamps = group_data.loc[(time_mask) & (group_data['Value'] == max_value), 'Timestamp'].tolist()

            daily_result['TimeRanges'].append({
                'TimeRange': label,
                'MaxValue': max_value,
                'Timestamps': max_timestamps
            })

        result.append(daily_result)

    return result


In [43]:
def showGlucoseLevels(closest_glucose_array,glucose_level_ts,glucose_level):
    levelsToShow = 48
    for glucose_ts in closest_glucose_array:
        indexOfGlucose = np.where(glucose_level_ts == glucose_ts)
        print("At glucose event " + str(glucose_ts) + " levels are: ")
        for i in range(levelsToShow):
            print(glucose_level[i + indexOfGlucose[0][0]])


In [44]:
def showGlucoseLevelsEvents(closest_glucose_array,glucose_level_ts,glucose_level):
    levelsToShow = 48
    for event_ts, glucose_ts in closest_glucose_array:
        indexOfGlucose = np.where(glucose_level_ts == glucose_ts)
        print("At meal/bolus event " + str(event_ts) + " levels are: ")
        for i in range(levelsToShow):
            print(glucose_level[i + indexOfGlucose[0][0]])
    

In [45]:
def replaceTimestampsForDates(result, dates_to_exclude):
    # Initialize a list to store the modified results
    modified_result = []

    # Iterate through each day's result
    for day_result in result:
        # Create a dictionary to store the modified time range results for this day
        modified_time_ranges = {}

        # Iterate through each time range result for this day
        for time_range_result in day_result['TimeRanges']:
            time_range_label = time_range_result['TimeRange']

            # Check if any date in the exclusion array falls within the time range of this day
            date_found = any(date in time_range_result['Timestamps'] for date in dates_to_exclude)

            if date_found:
                # Replace the timestamps with None
                timestamp_replacements = [None] * len(time_range_result['Timestamps'])
            else:
                timestamp_replacements = time_range_result['Timestamps']

            # Store the modified time range result
            modified_time_ranges[time_range_label] = {
                'Timestamps': timestamp_replacements
            }

        # Add the day's result with the modified time ranges to the modified result
        modified_result.append({
            'Date': day_result['Date'],
            'TimeRanges': modified_time_ranges
        })

    return modified_result


In [46]:
# Define your time ranges
time_ranges = {
    "6am-10am": (datetime.time(6, 0), datetime.time(10, 0)),
    "10am-2pm": (datetime.time(10, 0), datetime.time(14, 0)),
    "2pm-6pm": (datetime.time(14, 0), datetime.time(18, 0)),
    "6pm-10pm": (datetime.time(18, 0), datetime.time(22, 0))
}

def findTimestampsNotCoveredByMeals(result, meal_events):
    # Create a set of meal event timestamps for efficient lookup
    meal_events_set = set(meal_events)

    # Initialize a list to store timestamps
    timestamps_not_covered = []

    # Iterate through each day's result
    for day_result in result:
        # Iterate through each time range result for this day
        for time_range_result in day_result['TimeRanges']:
            time_range_label = time_range_result['TimeRange']
            timestamps = time_range_result['Timestamps']
            time_range_start, time_range_end = time_ranges[time_range_label]

            #print(timestamps)

           # Assuming meal is a datetime object and not just a time
            meal_events_within_range = False
            for meal in meal_events_set:
                meal_time = meal.time()
                time_range_start_datetime = datetime.datetime.combine(day_result['Date'], time_range_start)
                time_range_end_datetime = datetime.datetime.combine(day_result['Date'], time_range_end)

                if time_range_start_datetime <= meal <= time_range_end_datetime:
                    meal_events_within_range = True
                    break
        
            # Only add the timestamps if there are no meal events within the range
            if not meal_events_within_range:
                timestamps_not_covered.extend(timestamps)

    return timestamps_not_covered


In [47]:
data_points = []

In [48]:
glucose_timestamps = unfiltered['glucose_level_ts'].copy()#event_timestamps = interleave_arrays_increasing(unfiltered['meal_ts'].copy().to_numpy(), unfiltered['bolus_start_ts'].copy().to_numpy())

glucose_level_ts = pd.to_datetime(unfiltered['glucose_level_ts'].copy().to_numpy(), dayfirst=True, errors='coerce')
glucose_level = unfiltered['glucose_level'].copy().to_numpy()
bolus_start_ts = pd.to_datetime(unfiltered['bolus_ts'].copy().to_numpy(), dayfirst=True, errors='coerce')
meal_ts = pd.to_datetime(unfiltered['meal_ts'].copy().to_numpy(), dayfirst=True, errors='coerce')
bolus_dose = unfiltered['bolus_dose'].copy().to_numpy()

nan_mask = np.isnan(bolus_dose)
bolus_dose_filtered = bolus_dose[~nan_mask]

nat_mask = np.isnat(meal_ts)
meal_ts_filtered = meal_ts[~nat_mask]

nat_mask = np.isnat(bolus_start_ts)
bolus_start_ts_filtered = bolus_start_ts[~nat_mask]


#doses = interleave_arrays_increasing(bolus_start_ts_filtered,meal_ts_filtered)
closest_glucose_array_meals = glucoseForMealsTs(glucose_level_ts,meal_ts_filtered)

#########################

#meal_ts_filtered contains a numpy array containing all the meal timestamps with nat 

#Here, we find boluses that could potentially be valid and make them into a numpy array
bolusAndValueArray = bolusMealSeparation(meal_ts_filtered,bolus_start_ts_filtered,bolus_dose_filtered)

#afterwards we group them into morning,noon,evening etc. and use the max value bolus in order to find the timestamps
max_bolus_time_range = groupBolus2(bolusAndValueArray)

#Afterwards, we parse the meal and the bolus timestamps and we find look for meals that do not fit into any morning, noon,evening categories
# and if we can't, we replace them with the max value bolus
bolus_replacement_array = findTimestampsNotCoveredByMeals(max_bolus_time_range,meal_ts_filtered)

#Print separate
print("Meals")
print('\n'.join(map(str, meal_ts_filtered)))

print("Bolus")
print('\n'.join(map(str, bolus_replacement_array)))


#Interleave array increasingly by timestamp so we have a common timeline now
interleaved_meal_bolus_array = interleave_arrays_increasing(meal_ts_filtered,bolus_replacement_array)
print("Interleaved Meal Bolus")
print('\n'.join(map(str, interleaved_meal_bolus_array)))

#find the closest glucose_timestamp 
closest_glucose_meal_bolus_array = glucoseForEventsTs(glucose_level_ts,interleaved_meal_bolus_array)
print("closest glucose")
print(closest_glucose_meal_bolus_array)

#Using that timestamp, we then start from it and print 48 values (i.e. 4 hours of glucose recordings)
showGlucoseLevelsEvents(closest_glucose_meal_bolus_array, np.array(glucose_level_ts), np.array(glucose_level))

#### Run This!!!! 


Meals
2023-10-31 08:00:00
2023-10-31 12:18:00
2023-10-31 19:10:00
2023-10-31 19:34:00
2023-11-01 08:00:00
2023-11-01 11:01:00
2023-11-01 12:30:00
2023-11-01 15:45:00
2023-11-01 17:31:00
2023-11-01 19:45:00
2023-11-01 20:03:00
2023-11-01 20:20:00
2023-11-01 21:15:00
2023-11-02 07:37:00
2023-11-02 10:00:00
2023-11-02 12:30:00
2023-11-02 14:18:00
2023-11-02 14:18:00
2023-11-02 15:34:00
2023-11-02 18:30:00
2023-11-02 22:08:00
2023-11-03 08:15:00
2023-11-03 10:29:00
2023-11-03 12:09:00
2023-11-03 12:11:00
2023-11-03 14:03:00
2023-11-03 16:15:00
2023-11-03 17:26:00
2023-11-03 19:13:00
2023-11-03 23:21:00
2023-11-04 08:02:00
2023-11-04 10:13:00
2023-11-04 11:37:00
2023-11-04 11:56:00
2023-11-04 12:15:00
2023-11-04 17:40:00
2023-11-04 18:54:00
2023-11-04 20:42:00
2023-11-05 02:40:00
2023-11-05 08:15:00
2023-11-05 10:13:00
2023-11-05 10:13:00
2023-11-05 10:49:00
2023-11-05 17:34:00
2023-11-05 18:08:00
2023-11-06 04:37:00
2023-11-06 10:23:00
2023-11-06 15:03:00
2023-11-06 18:54:00
2023-11-06 21:

In [49]:
# Extract 'carbs_g' from the original data
carbs_ts = pd.to_datetime(unfiltered['meal_ts'], dayfirst=True)
carbs_g = unfiltered['carbs_g']
meal_tags = unfiltered['meal_tag']

In [50]:
data_points = []

# Iterate over each event timestamp and corresponding glucose timestamp
for event_ts, glucose_ts in closest_glucose_meal_bolus_array:
    # Assuming you have glucose_level_ts and glucose_level arrays containing timestamp and glucose level data respectively
    # Find the index of the closest value in the comparison array
    closest_index = np.abs(glucose_level_ts - glucose_ts).argmin()

    # Extract the glucose levels starting from the closest index for the next 48 timestamps
    glucose_levels = glucose_level[closest_index:closest_index + 48]

    # Extract the 'carbs_g' and 'meal_tag' values based on the timestamp (you need to have these arrays defined)
    carbs_value = carbs_g[closest_index] if closest_index < len(carbs_g) else None
    meal_tag_value = meal_tags[closest_index] if closest_index < len(meal_tags) else None

    data_point = {
        "EventTimestamp": event_ts,
        "GlucoseLevels": glucose_levels,
        "EventTag": carbs_value,  # Add the 'carbs_g' information
        "MealTag": meal_tag_value,  # Add the 'meal_tag' information
    }
    data_points.append(data_point)

# Create a DataFrame from the list of data points
GlucoseEvents = pd.DataFrame(data_points)

# Add day_of_the_week feature
GlucoseEvents['EventTimestamp'] = pd.to_datetime(GlucoseEvents['EventTimestamp'])
GlucoseEvents['day_of_the_week'] = GlucoseEvents['EventTimestamp'].dt.dayofweek


In [51]:
# Extract hour information
GlucoseEvents['hour'] = GlucoseEvents['EventTimestamp'].dt.hour

# Create a new column to categorize events into breakfast, lunch, or dinner
GlucoseEvents['MealCategory'] = pd.cut(
    GlucoseEvents['hour'],
    bins=[0, 10, 16, 22],
    labels=['Breakfast', 'Lunch', 'Dinner'],
    right=False
)

# Print the DataFrame
GlucoseEvents.head()

,EventTimestamp,GlucoseLevels,EventTag,MealTag,day_of_the_week,hour,MealCategory
0,2023-10-30 17:36:00,"[6.2, 7.1, 9.6, 5.1, 3.4, 4.6, 7.2, 8.3, 7.1, ...",NaN,NaN,0,17,Dinner
1,2023-10-31 08:00:00,"[7.8, 9.3, 11.1, 11.6, 10.6, 9.3, 8.3, 7.7, 6....",NaN,NaN,1,8,Breakfast
2,2023-10-31 12:18:00,"[5.3, 5.5, 6.3, 7.3, 7.4, 8.1, 8.3, 7.0, 5.7, ...",NaN,NaN,1,12,Lunch
3,2023-10-31 19:10:00,"[6.8, 7.1, 6.4, 7.0, 8.9, 9.8, 9.6, 9.7, 10.1,...",NaN,NaN,1,19,Dinner
4,2023-10-31 19:34:00,"[6.4, 7.0, 8.9, 9.8, 9.6, 9.7, 10.1, 10.2, 9.6...",NaN,NaN,1,19,Dinner


In [52]:
# Find peak of each glucose event
def find_peak_for_events(glucose_levels, event_timestamp):
    peak_index = np.argmax(glucose_levels)
    return peak_index
    

In [53]:
# Realign glucose events with peak at time zero and populate with data points from -75 to +150 minutes
def realign_events_with_peak(glucose_levels, event_timestamp):
    peak_index = find_peak_for_events(glucose_levels, event_timestamp)
    start_index = max(0, peak_index - 75)
    end_index = min(len(glucose_levels), peak_index + 151)
    realigned_glucose_levels = glucose_levels[start_index:end_index]
    return realigned_glucose_levels


In [54]:
def calculate_correlation(events):
    correlations = []
    for i in range(len(events)):
        for j in range(i+1, len(events)):  # Compare each event with all subsequent events
            correlation_coefficient, _ = pearsonr(events[i], events[j])  # Calculate Pearson correlation coefficient
            correlations.append(correlation_coefficient)
    return correlations
    

In [55]:
def prepare_data(events):
    prepared_data = []
    for idx, event in events.iteritems():
        prepared_data.append(event)  # Append the glucose levels directly
    return prepared_data

In [56]:
def multipass_clustering(events):
    # Prepare the data
    prepared_data = prepare_data(events)

    # Calculate correlations
    correlations = calculate_correlation(prepared_data)

    # Split based on MealCategory alone
    meal_category_groups = {}
    for idx, event in events.items():  # Use items() instead of iteritems()
        category = event[4]  # Accessing the category directly
        if category not in meal_category_groups:
            meal_category_groups[category] = []
        meal_category_groups[category].append((idx, event))  # Storing index along with event

    # Split based on MealTag alone
    meal_tag_groups = {}
    for idx, event in events.items():  # Use items() instead of iteritems()
        tag = event[3]  # Accessing the tag directly
        if tag not in meal_tag_groups:
            meal_tag_groups[tag] = []
        meal_tag_groups[tag].append((idx, event))  # Storing index along with event

    # Split based on MealCategory and MealTag
    meal_category_tag_groups = {}
    for idx, event in events.items():  # Use items() instead of iteritems()
        category_tag = (event[4], event[3])  # Accessing category and tag directly
        if category_tag not in meal_category_tag_groups:
            meal_category_tag_groups[category_tag] = []
        meal_category_tag_groups[category_tag].append((idx, event))  # Storing index along with event

    # Find the mean of each correlated group
    def find_group_mean(group):
        return np.mean([event[1][1] for event in group], axis=0)  # Accessing glucose levels from the second element of the tuple

    # Re-Mean groups and repeat step 5 of the multipass system
    def remean_groups(groups):
        means = {key: find_group_mean(group) for key, group in groups.items()}
        clustered_events = {}
        for key, group in groups.items():
            glucose_levels = [event[1] for event in group]  # Extract glucose levels from each event
            correlated_groups = [(correlation, event) for correlation, event in zip(calculate_correlation(glucose_levels), group)]
            correlated_groups.sort(reverse=True)  # Sort by correlation coefficient
            max_correlation_event = correlated_groups[0][1]
            if max_correlation_event not in clustered_events:
                clustered_events[max_correlation_event] = []
            clustered_events[max_correlation_event].append(group)


    # Initialize with the first pass
    clustered_events = remean_groups(meal_category_groups)
    clustered_events = remean_groups(meal_tag_groups)
    clustered_events = remean_groups(meal_category_tag_groups)

    return clustered_events

In [57]:
# Visualize each step of the multipass system for each of the groups with an interactive graph
def visualize_multipass_system(clustered_events):
    # Visualize each group
    for group, events in clustered_events.items():
        plt.figure(figsize=(10, 6))
        for event in events:
            plt.plot(event[0], event[1], label=f'Event {event[2]}')  # Plot realigned glucose events
        plt.xlabel('Time (minutes)')
        plt.ylabel('Glucose Level')
        plt.title(f'Group {group}')
        plt.legend()
        plt.grid(True)
        plt.show()


In [58]:
# Assuming you have isolated blood glucose events in a DataFrame called glucose_events
# glucose_events = pd.read_csv('blood_glucose_events.csv')

# Step 1: Find peak of each glucose event
GlucoseEvents['PeakIndex'] = GlucoseEvents.apply(lambda x: find_peak_for_events(x['GlucoseLevels'], x['EventTimestamp']), axis=1)

# Step 2: Realign glucose events with peak at time zero and populate with data points from -75 to +150 minutes
GlucoseEvents['RealignedData'] = GlucoseEvents.apply(lambda x: realign_events_with_peak(x['GlucoseLevels'], x['EventTimestamp']), axis=1)

# Step 3: Run Pearson’s correlation on all the realigned glucose events
correlations = calculate_correlation(GlucoseEvents['RealignedData'])

# Step 4: Multipass correlation-based clustering process
clustered_events = multipass_clustering(GlucoseEvents['RealignedData'])

# Visualize each step of the multipass system for each of the groups with an interactive graph
visualize_multipass_system(clustered_events)


/var/folders/9p/vwhkd5c50_j5cz3cyj0q4qsh0000gp/T/ipykernel_53728/1037179821.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, event in events.iteritems():


TypeError: unhashable type: 'numpy.ndarray'

#### Print the BG by Meal Type to excel 

In [ ]:
##  Split the GlucoseEvent dataframe into one for each meal

# Extract hour information
#GlucoseEvents['hour'] = GlucoseEvents['EventTimestamp'].dt.hour

# Create a new column to categorize events into breakfast, lunch, or dinner
#GlucoseEvents['meal_category'] = pd.cut(
#    GlucoseEvents['hour'],
#    bins=[0, 10, 16, 22],
#    labels=['Breakfast', 'Lunch', 'Dinner'],
#    right=False
#)

# Split the data into three dataframes based on meal category
#breakfast_data = GlucoseEvents[GlucoseEvents['meal_category'] == 'Breakfast']
#lunch_data = GlucoseEvents[GlucoseEvents['meal_category'] == 'Lunch']
#dinner_data = GlucoseEvents[GlucoseEvents['meal_category'] == 'Dinner']

In [ ]:
#def process_glucose_levels(glucose_data):
#    if isinstance(glucose_data, str):
#        glucose_values = glucose_data.strip('[]').split()
#    elif isinstance(glucose_data, np.ndarray):
#        glucose_values = glucose_data.tolist()
#    else:
#        raise TypeError("Unsupported data type for glucose_data")

#    glucose_values = [float(value) for value in glucose_values]
#    return glucose_values

#def write_to_excel(dataframe, output_file):
#    # Create a new DataFrame to hold the processed data
#    processed_data = pd.DataFrame(columns=['EventTimestamp', 'GlucoseLevel'])

#    # Iterate over rows in the input DataFrame
#    for index, row in dataframe.iterrows():
#        event_timestamp = row['EventTimestamp']
#        glucose_levels = process_glucose_levels(row['GlucoseLevels'])
        
#        # Append each glucose level with the timestamp to the processed DataFrame
#        for level in glucose_levels:
#            processed_data = processed_data.append({'EventTimestamp': event_timestamp, 'GlucoseLevel': level}, ignore_index=True)

#    # Write the processed DataFrame to Excel
#    processed_data.to_excel(output_file, index=False)

## Example usage:
## Replace GlucoseEventsBreakfast with your DataFrame variable and "GlucoseEventsBreakfast.xlsx" with your desired output file path
#write_to_excel(breakfast_data, "GlucoseEventsBreakfast.xlsx")

## Repeat the process for GlucoseEventsLunch and GlucoseEventsDinner DataFrames
## Replace GlucoseEventsLunch with your DataFrame variable and "GlucoseEventsLunch.xlsx" with your desired output file path
#write_to_excel(lunch_data, "GlucoseEventsLunch.xlsx")

## Replace GlucoseEventsDinner with your DataFrame variable and "GlucoseEventsDinner.xlsx" with your desired output file path
#write_to_excel(dinner_data, "GlucoseEventsDinner.xlsx")

#### Correlate Glucose Events

In [ ]:
# Extract hour information
GlucoseEvents['hour'] = GlucoseEvents['EventTimestamp'].dt.hour

# Create a new column to categorize events into breakfast, lunch, or dinner
GlucoseEvents['meal_category'] = pd.cut(
    GlucoseEvents['hour'],
    bins=[0, 10, 16, 22],
    labels=['Breakfast', 'Lunch', 'Dinner'],
    right=False
)


In [ ]:
# Split the data into three dataframes based on meal category
breakfast_data = GlucoseEvents[GlucoseEvents['meal_category'] == 'Breakfast']
lunch_data = GlucoseEvents[GlucoseEvents['meal_category'] == 'Lunch']
dinner_data = GlucoseEvents[GlucoseEvents['meal_category'] == 'Dinner']     
